# Comparison of Bayesian and Chi-Square Testing

## Objectives:
* Compare the Bayesian and Frequentist approach to solving a problem
* Observe that Bayesian is often able to reach a conclusion based on less data

Suppose there are two medicines under consideration to treat a disease. Medicine A is well-known to result in a 50% survival rate. Medicine B has not been studied. (Its true survival rate is 52%, but this is not known until we study it.)

We need to derive a test to determine if Medicine B is better than Medicine A and examine how much data we need to make this determination.

How would we approach this problem using the frequentist (Chi-Squared) technique and the Bayesian technique? How do these approaches work for various sample sizes?

In [ ]:
import numpy as np
'''Change or remove random seed to try this with different data'''
np.random.seed(123456)
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import beta, chisquare

'''The threshold value defines the baseline (or True) effectiveness
of the medicine being studied'''
threshold = 0.52

In [ ]:
sample_size = n = 50
''' B = outcomes of `n` patients who have been given medicne B
1 = survival, 0 = perish '''
B = 1*(np.random.random(n) < threshold)
print('The apparent effectiveness of this medicine (as measured from a small sample) is: ')
print(B.mean())

## First, the frequentist approach

Use the chi-squared test. If the p value is greater than 0.05, a frequentist would say that we can't reject the null hypothesis and therefore we continue to believe it, even if there is some reason to suspect the new medicine is better.

In [ ]:
survivals = B.sum()
deaths = n - survivals
chi_p_value = chisquare([deaths, survivals])[1]
chi_p_value

## Next, the Bayesian Approach

Rather than consider whether the null hypothesis can be rejected, we consider the proportion of time that Medicine B appears to be better than *status quo*.

In [ ]:
probability_B_better = (beta(1+survivals, 1+deaths).rvs(100_000)>.5).mean()
probability_B_better


## Let's see how this changes as sample size increases

let's suppose we have a large number of trial results, but we'll start by only considering a small subset of them.  As the size of this subset increases, we gain more confidence in our conclusions

In [ ]:
outcomes = 1*(np.random.random(100_000) < threshold)
print(f'A few outcomes: {outcomes[:20]}')
print(f'The average effectivenss, (as measured from a large sample set): {outcomes.mean()}')

In [ ]:
def comparison(n):
    trials = n['trials']
    trial_outcomes = outcomes[:trials]
    survivals = trial_outcomes.sum()
    deaths = trials - survivals
    frequentist_p = chisquare([deaths, survivals])[1]
    bayesian_prob = (beta(1+survivals, 1+deaths).rvs(100_000)>.5).mean()
    return pd.Series({'survivals':survivals, 
                      'deaths':deaths, 
                      'p':frequentist_p, 
                      'bayes':bayesian_prob})

In [ ]:
df = pd.DataFrame({'trials':np.unique(np.logspace(2,5, 200).astype(np.int))}, dtype = np.int)
x = pd.concat([df, df.apply(comparison, axis = 1)], axis = 1).set_index('trials')
x.head(20)

In [ ]:
plt.plot(1-x['p'], label = '1 - Chi Square p_value')
plt.plot(x['bayes'], label = 'Bayes')
plt.axhline(.95)
plt.xscale('log')
plt.xlabel('Number of Trials')
plt.ylabel('Confidence or Probability of Superior Performance')
plt.title('Comparison of Bayes and Frequentist Tests')
plt.legend();

This shows us that both the Chi Square test and the Bayesian test approach a high level of certainty for large sample sizes.
Note that Bayes is nearly always above the Frequentist, and has less variation. Generally, Bayes will exceed the 95% threshold sooner.

How much sooner is Bayes arriving at that conclusion?

In [ ]:
print('First time Bayes exceeded 95%: ')
print(x[x['bayes'] > .95].index.min(),' trials')

In [ ]:
print('First time Chi Squared exceeded 95% (p<.05): ')
print(x[x['p'] < .05].index.min(),' trials')

In [ ]:
print('Last time Bayes was less than 95%')
print(x[x['bayes'] < .95].index.max(), ' trials')

In [ ]:
print('Last time Chi Squared was less than 95% (p>.05): ')
print(x[x['p'] > .05].index.max(), ' trials')

# Conclusion: 

### While both Bayesian and Frequentist approaches were able to arrive at the correct result, Bayes was able to reach that result with several hundred, or several thousand fewer trials.

### In the real world, this could save time, money, effort and possibly even lives.

## Explore:

* Try changing the np.random.seed() value in the first cell (or remove this statement) to try this with different values
* Try changing the threshold value to see how this changes when the effect size changes.
* If threshold = 0.5, the new medicine, B, should be identical to the *status quo* medicine.